In [29]:
import requests


class NflApi:
     
    def get_data_from_url(self, url: str) -> dict:
        """Gets data from specified URL and returns in form of dict
        raises exception otherwise.

        Args:
            url (str): API url to get data

        Returns:
            dict: Returns dict of the data from URL.
        """
        response = requests.get(url)
        try:
            return response.json()
        except:
            print('Error')

TEAMS_IDS = f'https://site.api.espn.com/apis/site/v2/sports/football/nfl/teams'

#TODO add a dynamic way to update this url
ATHLETE_IDS = f'https://site.api.espn.com/apis/site/v2/sports/football/nfl/teams/12/roster'





In [36]:
teams_data =  NflApi().get_data_from_url(url = TEAMS_IDS)


#TODO make api use generators for further enhancement, make url dynamic for different values
def get_game_ids(data: list[dict])-> list[tuple]:
    num_of_teams = len(teams_data['sports'][0]['leagues'][0]['teams'])
    
    return [(teams_data['sports'][0]['leagues'][0]['teams'][i]['team']['id'], \
             teams_data['sports'][0]['leagues'][0]['teams'][i]['team']['displayName']) \
            for i in range(num_of_teams)]
    
#TODO seperate api call, transformation & load into separate files, 
import pandas as pd

team_ids_df = pd.DataFrame(get_game_ids(teams_data), 
                           columns=['team_id', 'team_name'])

In [39]:
athlete_data =  NflApi().get_data_from_url(url = ATHLETE_IDS)

team_rosters = []

# Iterate through team_ids in teams_df
for team_id, team_name in zip(team_ids_df['team_id'], team_ids_df['team_name']):
    # Make API request
    api_url = f'https://site.api.espn.com/apis/site/v2/sports/football/nfl/teams/{team_id}/roster'
    response = requests.get(api_url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        roster_data = response.json()
        
        # Store the roster data in the dictionary with team_id as the key
        num_of_athletes = len(roster_data['athletes'][0]['items'])
        
        for i in range(num_of_athletes):
            player_name = roster_data['athletes'][0]['items'][i]['fullName']
            team_rosters.append({'team_name': team_name, 'player_name': player_name})
    else:
        print(f"Failed to fetch data for {team_name} (Team ID: {team_id})")

# Create a DataFrame from the list of dictionaries
team_player_df = pd.DataFrame(team_rosters)